In [ ]:
import os
from typing import Tuple
from llm_orchestration import *
from nebula3_database.movie.movie_db import MOVIE_DB
from experts.pipeline.api import PipelineApi, PipelineTask

def test_pipeline_task(pipeline_id):
    class LlmTask(PipelineTask):
        def __init__(self):
            self.llm_task = LlmTaskInternal()
            print("LlmTask Initialized successfully.")

        def process_movie(self, movie_id: str) -> Tuple[bool, str]:
            print (f'LlmTask: handling movie: {movie_id}')

            output = self.llm_task.process_movie(movie_id)

            print("LlmTask: Finished handling movie.")
            print(output)
            return output
        def get_name(self) -> str:
            return "llm"

    pipeline = PipelineApi(None)
    task = LlmTask()
    pipeline.handle_pipeline_task(task, pipeline_id, stop_on_failure=True)


In [ ]:
ppl = PipelineApi(None)

In [ ]:
ppl.get_new_movies("12345678","llm")

In [ ]:
test_pipeline_task("12345678")

In [ ]:
task = LlmTaskInternal()

In [ ]:
mid = MovieImageId("Movies/-3103202934810463453",90)
mobj = task.nebula_db.get_movie_frame_from_collection(mid)

In [ ]:
mobj['url']

In [ ]:
rc = task.process_target_id(mid,image_url=mobj['url'],n=5)

In [ ]:
rc

In [ ]:
task.nebula_db.write_movie_frame_doc_to_collection(mid,rc,LLM_OUTPUT_COLLECTION)

In [ ]:
task.process_movie("Movies/-3103202934810463453",n=5)